# Sesión 10
> Por Christian Rubio Montiel (CRM)
Implementación por Moisés Borjan Ramírez, Josue Río Campos Becerra, Oscar David Domínguez Dávila, Luis Ramos Guerrero y Kevin Martin Rivera Castro y CRM

En esta sesión se introduce una nueva estructura de datos y se de otra solución óptima al problema de ordenación mediante el algoritmo llamado Heapsort. Se aborda el uso de instrucciones en Julia: [Instrucciones](https://www.dropbox.com/s/ll1ke40q7wxt54a/Instrucciones.ipynb?dl=0).

<a id='indice'></a>
### Índice
---
1. **[Max-heaps](#Max-heaps)**
2. **[MAX-HEAPIFY](#Max-heapify)**
3. **[Referencias](#referencias)**
---

El algoritmo de Heapsort combina características de Insertion y Merge Sort, además de introducir otra técnica de diseños de algoritmos a través de una estructura de datos llamada heap (montículo) que es un árbol binario.

Dado un arreglo $A$, por ejemplo, $A=[16,14,10,8,7,9,3,2,4,1]$ las posiciones de los elementos van del $1$ al $10$. Con fines didácticos, escribiremos como subíndice la posición del número cuando haga falta, por ejemplo, 
$A=[16_1,14_2,10_3,8_4,7_5,9_6,3_7,2_8,4_9,1_{10}]$.

En Julia, escribimos en arreglo $A$ como sigue, y podemos saber el número en la posición $i$ al ejecutar $A[i]$.

In [1]:
A=[16,14,10,8,7,9,3,2,4,1];

In [2]:
A[1]

16

Un *árbol binario* es un árbol de $n$ vértices, con un vértice distinguido $v$ llamado *raíz*, el cual tiene a lo más dos vecinos, los cuales se llaman *hijo derecho* e *hijo izquierdo*; estos a su vez, tienen a lo más dos vecinos, los cuales se llaman *hijo derecho* e *hijo izquierdo* y así sucesivamente.

En esta parte, trabajaremos con los *árboles binarios completos* que son aquellos árboles binarios donde $|N^i(v)|=2^i$ donde $1\leq i < k$ con $k$ el radio del árbol y $N^k(v)$ se forma por hojas que se llena de izquierda a derecha.

Vamos a acomodar el arreglo $A$ en esta estructura de datos:

<img src="Fig101.png" alt="Figura 5" width="600"/>

> `Figura 5: Arreglo visto como un árbol binario completo, ver [2]. A la izquierda del árbol se indican las alturas h_i y a la derecha del árbol se indican la i-ésima vecindad N^i(v) de la raíz v.`

Entonces, cada vértice del árbol corresponde a un elemento del arreglo. Debajo de cada vértice se indica la posición correspondiente en el arreglo. La raíz $v=N^0(v)$ es el elemento de la primer posición. Su vecindad $N(v)=N^1(v)$ son los 2 hijos de $v$, su hijo izquierdo corresponde al elemento del de la posición 2, su hijo derecho corresponde al elemento del de la posición 3. Su segunda vecindad $N^2(v)$ está forma por 4 vértices, con índices del 4 al 7. En general, cada vecindad tendrá el doble de elementos que la vecindad anterior, salvo posiblemente los vértices en la última vecindad. Para verificar esto, podemos establecer una biyección de los vértices en la $i$-ésima vecindad con los hijos izquierdos, y una biyección entre los hijos izquierdos con los derechos.

Entonces, si el radio del árbol es $k$, la $i$-ésima vecindad (también llamada el *$i$-ésimo nivel*) tendrá $2^i$ vértices si $0\leq i < k$, y $2^k\leq n < 2^{k-1}$, por lo que $k\leq \lg(n) < k+1 $ y entonces $k=\left\lfloor \lg(n) \right\rfloor$. En el ejemplo, $\left\lfloor \lg(10) \right\rfloor=\left\lfloor 3.3... \right\rfloor=3$.

Por esta razón, si un vértice $u$ tiene índice $j$ (la posición $j$-ésima del arreglo), la distancia de la raíz a $u$ es $\left\lfloor \lg(j) \right\rfloor$.

In [3]:
log2(10)

3.321928094887362

Dependiendo del texto, se pueden dar distintas nomenclaturas. Aquí definimos la altura $h$ del árbol como $k+1$, los vértices a altura $i$ serán quellos en el nivel $k+1-i$, denotada $h_i$.

Observemos lo siguiente. La raíz está en la posición $1$, su hijo izquierdo está en la posición $2$, el cual, a su vez está en la posición $2^2$ su hijo izquierdo. Entonces, el primer vértice en el $i$-ésimo nivel está en la posición $1+2+2^2+...+2^{i-1}+1=2^i-1+1=2^i$.

Esto nos indica que un vértice en el $i-1$-ésimo con un índice $2^{i-1}+l$ ($0\leq l < 2^{i-1}$), su hijo izquierdo estará en la posición $2^i+2l=2(2^{i+1}+l)$. Dicho de otro modo, cualquier vértice en la posición $j$, su hijo izquierdo está en la posición $2j$ y su hijo derecho está en la posición $2j+1$ siempre que $2j\leq n$, o bien, $2j+1\leq n$.

Regresar al **[Índice](#indice)**.

<a id='Max-heaps'></a>
## 1. Max-heaps

Para movernos sobre el arreglo $A$ de tal forma que la estructura de datos es un árbol completo, requerimos de los siguientes seudocódigos que reciben de entrada un índice $i$ del arreglo, con $1\leq i \leq n$.

*LEFT($i$)*
1. **return** $2i$

*RIGHT($i$)*
1. **return** $2i+1$

*PARENT($i$)*
1. **return** $\left\lfloor \frac{i}{2}\right\rfloor$

Claramente, cada una de estas rutinas son de tiempo constante $\theta(1)$, y son correctas por la explicación previa.

In [4]:
function LEFT(i)
    return 2*i
end

function RIGHT(i)
    return 2*i+1
end

function PARENT(i)
    return div(i,2)
end

PARENT (generic function with 1 method)

Decimos que un arreglo es *max-heap* si para cada vértice $A[PARENT(i)]\geq A[i]$, lo que implica que el elemento en la raíz será el máximo del arreglo.

Decimos que un arreglo es *min-heap* si para cada vértice $A[PARENT(i)]\leq A[i]$, lo que implica que el elemento en la raíz será el mínimo del arreglo.

El árbol de la Figura 5 es un ejemplo de un max-heap, sin embargo, el arreglo $A$ no está ordenado.

Regresar al **[Índice](#indice)**.

<a id='Max-heapify'></a>
   ## 2. MAX-HEAPIFY

En la técnica de *divide y vencerás* usada en el algoritmo de MAX-HEAP, el mezclado de dos árboles binarios enraizados completos que son max-heaps y que sus respectivas raíces comparten el mismO padre se da en la siguiente rutina.

En este algoritmo, la entrada es un arreglo $A$ de $n$ números y un índice $i$ de un vértice cuyos hijos son las raíces de max-heaps. La salida será el arreglo $A$ tal que del índice $i$ es la raíz de un max heap.

*MAX-HEAPIFY($A$,$i$)*
1. $l=LEFT(i)$
2. $r=RIGHT(i)$
3. **if** $l\leq heapSize[A]$ and $A[l]>A[i]$ **then**
4. $\hspace{0.6cm}largest=l$
5. $\hspace{0.6cm}$**else** $largest=i$
6. **if** $r\leq heapSize[A]$ and $A[r]>A[largest]$ **then**
7. $\hspace{0.6cm}largest=r$
8. **if** $largest \not = i$ **then**
9. $\hspace{0.6cm}$exchange($A[i],A[largest]$)
10. $\hspace{0.6cm}$MAX-HEAPIFY($A$,$largest$)

Ver que el algoritmo es correcto, es relativamente sencillo. El caso base es trivial, ya que un árbol trivial es max-heap. En las primeras nueve líneas, se verifica cuál es el elemento mayor entre el padre $i$ y sus hijos. El loop invariante se verifica puesto que por hipótesis de inducción el índice $largest$ está en un árbol de menor altura, devolviendo así un max-heap.

El tiempo de ejecución también es relativamente sencillo calcularlo. Como el índice $i$ se detiene en la recursión cuando $largest$ es una hoja, así que en el peor de los casos se recorre la altura del árbol, por lo que el algoritmo tiene un tiempo de ejecución $O(\lg(n))$.

In [5]:
function MAX_HEAPIFY(A,i,heapSizeA)
    l = LEFT(i)
    r = RIGHT(i)
    if l <= heapSizeA && A[l] > A[i]
        largest = l
    else
        largest = i
    end
    if r <= heapSizeA && A[r] > A[largest]
        largest = r
    end
    if largest != i
        A[i], A[largest] = A[largest], A[i]
        MAX_HEAPIFY(A,largest,heapSizeA)
    end
end

MAX_HEAPIFY (generic function with 1 method)

Regresar al **[Índice](#indice)**.

<a id='referencias'></a>
## 3. Referencias

$[1]$ Cormen, T. H., Leiserson, C. E., Rivest, R. L. y Stein C. (2022). **Introduction to algorithms**. MIT Press, 4E.